# 1. 필요 모듈 임포트

In [253]:
#필요 모듈 임포트
from google.cloud import texttospeech
from google.cloud import speech
from pydub import AudioSegment
from konlpy.tag import Okt
from konlpy.tag import Kkma
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import matplotlib.pyplot as plt
import FinanceDataReader as fdr
import speech_recognition as sr
import pandas_datareader.data as pdr
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import matplotlib
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima.arima import auto_arima
from tqdm import tqdm
import seaborn as sns
import itertools
import pandas as pd
import numpy as np
import datetime
import subprocess
import openai
import time
import os
import io
import warnings

warnings.filterwarnings("ignore")
plt.style.use('seaborn-whitegrid')
%matplotlib inline

# 2. 데이터 로드 및 안내 음성 파일 생성

In [275]:
#상장 기업 목록 가져오기
krx = pd.read_csv('/Users/parkjunhyeong/Desktop/박준형/02. 대내 및 대외활동/02. 대외활동/03. 신한AI 공모전/01. 소스/01. 데이터/KRX_data.csv')
companies = krx['Name'].tolist()

#open ai api key
openai.api_key = "  "

#크롬 드라이버 경로 설정
path = "/Users/parkjunhyeong/Downloads"

#오늘의 뉴스 요약
df_naver = naverCrawling(path)
summary1 = newsSummarize(df_naver['본문'])

#안내 멘트 음성 파일 생성
synthesizeText("안녕하세요. 고객님의 투자를 도와주는 신한 AI입니다.", "./Ment1.wav")
synthesizeText("원하시는 서비스를 말씀해주세요.", "./Ment2.wav")
synthesizeText("계속해서 신한 AI 서비스를 이용하시려면 예, 서비스를 종료하시려면 아니오라고 말씀해주세요.", "./Ment3.wav")
synthesizeText("다음 네 가지 서비스 중 원하는 것을 말씀해주세요. 1번 최근 기업 뉴스 요약 서비스. 2번 재무 정보 요약 서비스. 3번 주가 추세 분석 서비스. 4번 주가 예측 서비스.", "./Ment4.wav")
synthesizeText("원하시는 종목을 말씀해주세요.", "./Ment5.wav")
synthesizeText("말씀하신 종목을 찾을 수 없습니다.", "./Ment6.wav")
synthesizeText("원하시는 재무정보를 말씀해주세요.", "./Ment7.wav")
synthesizeText("원하시는 정보를 불러오고 있습니다. 잠시만 기다려주세요.", "./Ment8.wav")
synthesizeText("말씀하신 내용을 이해할 수 없습니다.", "./Ment9.wav")
synthesizeText("신한 AI 서비스를 이용해주셔서 감사합니다.", "./Ment10.wav")
synthesizeText("음성 정보를 생성하고 있습니다. 잠시만 기다려주세요.", "./Ment11.wav")
synthesizeText("말씀하신 종목의 투자정보 서비스를 계속 이용하시려면 예, 그만 이용하시려면 아니오라고 말씀해주세요.", "./Ment12.wav")

# 3. 필요 함수 정의

## 3-1. Text To Speech API 사용하는 함수

In [276]:
#text to speech api 사용하는 함수
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './tts_key.json' #경로 자기에 맞게 설정

def synthesizeText(text, file_name):
    client = texttospeech.TextToSpeechClient()

    input_text = texttospeech.SynthesisInput(text=text)

    voice = texttospeech.VoiceSelectionParams(
        language_code='ko-KR',
        ssml_gender=texttospeech.SsmlVoiceGender.NEUTRAL)

    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.LINEAR16)

    response = client.synthesize_speech(
        input=input_text,
        voice=voice,
        audio_config=audio_config)

    with open(file_name, 'wb') as out:
        out.write(response.audio_content)

## 3-2. 음성 데이터 입력받고 텍스트로 변환하는 함수

In [277]:
#사용자로부터 음성 데이터를 입력받아 텍스트로 변환하여 반환하는 함수
def voiceRecog(num):
    r = sr.Recognizer()

    with sr.Microphone() as source:
        print('음성을 입력하세요.')
        audio = r.record(source, duration = num)
        
        print('음성변환 : ' + r.recognize_google(audio, language='ko-KR'))
        return r.recognize_google(audio, language='ko-KR')

## 3-3. 네이버 증권 주요 뉴스 크롤링하는 함수

In [278]:
def naverCrawling(path):
    # Selenium 드라이버 실행
    options = webdriver.ChromeOptions()
    options.add_argument('headless')  # Chrome 창 숨기기 옵션
    driver = webdriver.Chrome(path + '/chromedriver', options=options) #크롬 드라이버 경로 자기한테 맞게 변경

    # 네이버 증권 뉴스 페이지로 이동
    driver.get('https://finance.naver.com/news/mainnews.naver')

    # 페이지 로딩을 위해 3초 대기
    time.sleep(3)

    # 페이지 소스코드 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 뉴스 제목 추출하기
    src = soup.select('.articleSubject')
    news_titles  = [news.text for news in src]
    news_titles = [news.replace("\n", "") for news in news_titles]


    #뉴스 링크 추출하기
    news_links = []
    base_url = 'https://finance.naver.com/'

    for s in src:
        end = str(s.a).find(">")
        link = str(s.a)[9:end-1]
        link = base_url + link.replace("amp;","")
        news_links.append(link)

    #뉴스 내용 추출하기
    news_contents = []
    for link in news_links:
        driver.get(link)
        time.sleep(2)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        content = soup.select('.articleCont')[0].text.strip()
        news_contents.append(content)

    # 드라이버 종료
    driver.quit()

    #시장 전반 뉴스 데이터 프레임 형태로 저장하기
    df = pd.DataFrame(columns = ['제목', '링크', '본문'])
    df['제목'] = news_titles
    df['링크'] = news_links
    df['본문'] = news_contents
    
    #결과 데이터프레임 반환
    return df

## 3-4. 주요 뉴스 요약 함수 (chatGPT api 활용)

In [279]:
def newsSummarize(text):
    question = f"다음 기사들 각각 3줄 이내로 요약해서, 순서대로 앵커같은 말투로 바꿔줘:\n\n {text}"

    completion = openai.ChatCompletion.create(
      model = "gpt-3.5-turbo",
      messages=[
        {"role": "user", "content": question}
      ]
    )
    
    response = "오늘의 주요 증권 기사 요약입니다.\n"
    response += completion.choices[0].message.content

    return response

## 3-5. 종목 뉴스 요약 함수

In [280]:
def companyNewsSummarize(df, company):
    df11 = df
    summarized = []

    for i in tqdm(range(3)):
        summarize = newsSummarize(df11[i])
        summarized.append(summarize)

    ##2nd summarization
    question = f"다음 기사들 중 {company} 관련 내용만을 요약해서, 순서대로 앵커같은 말투로 바꿔줘:\n\n {summarized}"

    completion = openai.ChatCompletion.create(
      model = "gpt-3.5-turbo",
      messages=[
        {"role": "user", "content": question}
      ]
    )
    
    response = "오늘의 " + company + "관련 뉴스 요약입니다.\n"
    response += completion.choices[0].message.content
    return response

## 3-6. 종목 분석 리포트 요약 함수

In [281]:
def companyReportSummarize(df, company):
    df22 = df
    summarized = []

    for i in tqdm(range(3)):
        summarize = newsSummarize(df22[i])
        summarized.append(summarize)

    ##2nd summarization
    question = f"다음 기사들 중 {company} 관련 내용만을 요약해서, 순서대로 앵커같은 말투로 바꿔줘:\n\n {summarized}"

    completion = openai.ChatCompletion.create(
      model = "gpt-3.5-turbo",
      messages=[
        {"role": "user", "content": question}
      ]
    )
    
    response = "오늘의 " + company + "관련 분석 요약입니다.\n"
    response += completion.choices[0].message.content
    
    return response

## 3-7. Investing.com 종목별 주요 뉴스 크롤링하는 함수

In [282]:
def crawlingInv(path, company):
    # Selenium 드라이버 실행
    options = webdriver.ChromeOptions()
    options.add_argument('headless')  # Chrome 창 숨기기 옵션
    driver = webdriver.Chrome(path + '/chromedriver') #크롬 드라이버 경로 자기한테 맞게 변경

    # 웹 드라이버 경로 설정
    driver_path = path + '/chromedriver'

    # Selenium 드라이버 실행
    driver = webdriver.Chrome(driver_path)

    # kr.investing.com/equities 사이트 접속
    driver.get('https://kr.investing.com/equities')

    name = company

    # 광고 팝업 닫기
    try:
        popup_close_button = WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'i.popupCloseIcon.largeBannerCloser'))
        )
        popup_close_button.click()
    except:
        pass

    # 검색 입력란에 회사 이름 입력 후 검색 버튼 클릭
    search_box = WebDriverWait(driver, 2).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.searchText.arial_12.lightgrayFont.js-main-search-bar'))
    )
    search_box.send_keys(name)

    search_button = WebDriverWait(driver, 2).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.searchGlassIcon.js-magnifying-glass-icon'))
    )
    search_button.click()

    # 광고 팝업 닫기
    try:
        popup_close_button = WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'i.popupCloseIcon.largeBannerCloser'))
        )
        popup_close_button.click()
    except:
        pass

    # 대기 시간 늘리기 및 다른 선택자 사용
    article_items = WebDriverWait(driver, 3).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.articleItem'))
    )

    news_titles = []
    news_links = []
    news_contents = []

    for item in article_items:
        # Extract article title
        title_element = WebDriverWait(item, 3).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'a.title'))
        )
        news_titles.append(title_element.text)

        # Extract article link
        news_links.append(title_element.get_attribute('href'))


    # Loop through the news links and scrape the content for each one
    for link in news_links:
        # Go to the page you want to scrape
        driver.get(link)

        # Wait for the page to load and find the content element
        content_element = WebDriverWait(driver, 4).until(EC.presence_of_element_located((By.XPATH, "//div[contains(@class,'WYSIWYG articlePage')]")))

        # Extract the text from the content element
        content = content_element.text
        content = content.replace('\n','').rsplit('.', 1)[0]
        news_contents.append(content)

    # 드라이버 종료
    driver.quit()

    df = pd.DataFrame(columns = ['제목', '링크', '본문'])
    df['제목'] = news_titles
    df['링크'] = news_links
    df['본문'] = news_contents
    
    df1 = df.iloc[0:3]  # 1번째 행부터 3번째 행 추출
    df11 = df1['본문'].values.tolist()

    df2 = df.iloc[3:6]  # 4번째 행부터 6번째 행 추출
    df22 = df2['본문'].values.tolist()

    return df11, df22

## 3-8. KRX 정보데이터 시스템에서 개별 종목 관련 정보 크롤링하는 함수

In [283]:
def financeCrawling(path, company):
    # 정보를 가져오고 싶은 종목명 입력
    name = company

    # Selenium 드라이버 실행
    options = webdriver.ChromeOptions()
#     options.add_argument('headless')  # Chrome 창 숨기기 옵션

    # 웹 드라이버 경로 설정
    driver_path = path + '/chromedriver'

    # Selenium 드라이버 실행
    driver = webdriver.Chrome(driver_path)

    # KOREA EXCHANGE 주식 종목 사이트 접속
    driver.get('http://data.krx.co.kr/contents/MMC/ISIF/isif/MMCISIF001.cmd')

    # 버튼을 찾아 클릭
    button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[@class='btnSearch CI-STOCK-SEARCH-OPEN']")))
    button.click()

    # 검색 입력란에 회사 이름 입력
    search_input = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'jsStockSearchLayerWord')))
    search_input.clear()
    search_input.send_keys(name)

    # 엔터 입력
    time.sleep(3)
    search_input.send_keys(Keys.RETURN)
    time.sleep(5)
    search_input.send_keys(Keys.RETURN)

    # 페이지 로딩을 위해 10초 대기
    time.sleep(5)

    # HTML 요소를 찾기 위해 JavaScript 실행
    element_script = "document.querySelector('a[data-index=\"0\"]').click();"
    driver.execute_script(element_script)

    # 페이지 로딩을 위해 5초 대기
    time.sleep(5)

    # 요소의 값을 가져와 변수에 저장
    sb_prc = driver.find_element(By.CSS_SELECTOR, 'td[data-bind="SB_PRC"]').text.replace(',', '')
    date = datetime.datetime.now().strftime('%Y-%m-%d')
    company = driver.find_element(By.CSS_SELECTOR, 'strong[data-bind="ISU_NM"]').text
    forn_rto = driver.find_element(By.CSS_SELECTOR, 'td[data-bind="FORN_RTO"]').text
    tdd_opnprc = driver.find_element(By.CSS_SELECTOR, 'td[data-bind="TDD_OPNPRC"]').text.replace(',', '')
    acc_trdvol = driver.find_element(By.CSS_SELECTOR, 'td[data-bind="ACC_TRDVOL"]').text.replace(',', '')
    tdd_hgprc = driver.find_element(By.CSS_SELECTOR, 'td[data-bind="TDD_HGPRC"]').text.replace(',', '')
    acc_trdval = driver.find_element(By.CSS_SELECTOR, 'td[data-bind="ACC_TRDVAL"]').text.replace(',', '')
    tdd_lwprc = driver.find_element(By.CSS_SELECTOR, 'td[data-bind="TDD_LWPRC"]').text.replace(',', '')
    mktcap = driver.find_element(By.CSS_SELECTOR, 'td[data-bind="MKTCAP"]').text.replace(',', '')
    wk52_hgst_prc = driver.find_element(By.CSS_SELECTOR, 'td[data-bind="WK52_HGST_PRC"]').text.replace(',', '')
    wk52_lwst_prc = driver.find_element(By.CSS_SELECTOR, 'td[data-bind="WK52_LWST_PRC"]').text.replace(',', '')
    forn_rto = driver.find_element(By.CSS_SELECTOR, 'td[data-bind="FORN_RTO"]').text.replace(',', '')
    per_pbr = driver.find_element(By.CSS_SELECTOR, 'td[data-bind="PER_PBR"]').text.replace(',', '')
    sb_prc = driver.find_element(By.CSS_SELECTOR, 'td[data-bind="SB_PRC"]').text.replace(',', '')
    div_yd = driver.find_element(By.CSS_SELECTOR, 'td[data-bind="DIV_YD"]').text.replace(',', '')
    asst_totamt = driver.find_element(By.CSS_SELECTOR, 'span[data-bind="ASST_TOTAMT"]').text.replace(',', '')
    debt_totamt = driver.find_element(By.CSS_SELECTOR, 'span[data-bind="DEBT_TOTAMT"]').text.replace(',', '')
    cap = driver.find_element(By.CSS_SELECTOR, 'span[data-bind="CAP"]').text.replace(',', '')
    cap_grndtot = driver.find_element(By.CSS_SELECTOR, 'span[data-bind="CAP_GRNDTOT"]').text.replace(',', '')
    sales = driver.find_element(By.CSS_SELECTOR, 'span[data-bind="SALES"]').text.replace(',', '')
    operproft_amt = driver.find_element(By.CSS_SELECTOR, 'span[data-bind="OPERPROFT_AMT"]').text.replace(',', '')
    netincm = driver.find_element(By.CSS_SELECTOR, 'span[data-bind="NETINCM"]').text.replace(',', '')

    # 드라이버 종료
    driver.quit()

    # DataFrame 데이터 추가
    data = {'날짜': [date], '회사': [company], '시가': [tdd_opnprc], '거래량': [acc_trdvol],
            '고가': [tdd_hgprc], '거래대금': [acc_trdval], '저가': [tdd_lwprc], '시가총액': [mktcap],
            '52주 최고 종가': [wk52_hgst_prc], '52주 최저 종가': [wk52_lwst_prc], '외국인비율': [forn_rto],
            'PER/PBR': [per_pbr], '대용가': [sb_prc], '배당수익률': [div_yd], '자산총계': [asst_totamt],
            '부채총계': [debt_totamt], '자본금': [cap], '자본총계': [cap_grndtot], '매출액': [sales],
            '영업이익': [operproft_amt], '당기순이익': [netincm]}

    df_stock = pd.DataFrame(data)
    
    col_list = ['시가', '거래량', '고가', '거래대금', '저가', '시가총액', '52주 최고 종가','52주 최저 종가','대용가','자산총계', '부채총계', 
            '자본금','자본총계', '매출액', '영업이익', '당기순이익']

    for col in col_list: 
        df_stock[col] = numberToKorean(int(df_stock[col][0]))

    return df_stock

## 3-9. 금액을 api가 읽을 수 있는 한국어로 바꿔주기 위한 함수

In [284]:
#금액 숫자를 text to speech api가 읽을 수 있는 한국어로 바꿔주기 위한 함수 정의
def numberToKorean(num):
    units = ['', '만', '억', '조', '경']
    count = 0
    result = ''
    while num > 0:
        num, mod = divmod(num, 10000)
        if mod > 0:
            result = '{}{}{}'.format(mod, units[count], result)
        count += 1
    return result

## 3-10. 재무정보 데이터 프레임을 문자열로 바꾸어 주는 함수

In [285]:
def financeString(df, text):
    text = text.replace(" ", "")
    text = text.lower()
    
    col_list = ['시가', '거래량', '고가', '거래대금', '저가', '시가총액', '52주 최고 종가','52주 최저 종가','대용가','자산총계', '부채총계', 
            '자본금','자본총계', '매출액', '영업이익', '당기순이익']
    
    response_str = '{}년 {}월 {}일 {}의 주식 정보에 대해 안내해 드리겠습니다.\n'.format(df['날짜'][0][:4],
                                                                    df['날짜'][0][5:7],
                                                                    df['날짜'][0][8:],
                                                                    df['회사'][0])
    for col in col_list:
        col_n = col.replace(" ", "")
        if  col_n in text:
            response_str += "{} : {}원.\n".format(col, df[col][0])

    if 'per' in text:
        response_str += "PER: {}.\n".format(df['PER/PBR'][0].split('/')[0])
    if 'pbr' in text:
        response_str += "PBR: {}.\n".format(df['PER/PBR'][0].split('/')[1])
        print('in pbr')
    if '배당' in text:
        response_str += "배당수익률은 {}%".format(df['배당수익률'][0])
        
    response_str += "입니다."
    
    return response_str

## 3-11. 데이터 평활을 통해 주가 추세 요약 데이터 프레임 생성하는 함수

In [286]:
def chartInfo(company, m = 6):
    # 종목 코드 설정
    code = krx[krx['Name'] == company]['Code'].values[0]

    # 시작 날짜 설정 (현재 날짜로부터 6달 이전)
    # 사용자로부터 개월 입력받을 수 있도록 추후 설정 예정
    month = m
    date = datetime.datetime.now() - datetime.timedelta(days=month*30)

    # 주가 데이터 가져오기
    df = fdr.DataReader(code, date)

    # 'Date'를 열로 가져오기
    df['Date'] = df.index

    # 'Close'를 제외한 열 제거
    df = df[['Date','Close']]

    # 이동평균(Moving Average) 계산 (7영업일 기준)
    win = 7 
    df['MA'] = df['Close'].rolling(window=win).mean()

    # 이동평균(Moving Average)를 다시 적용하여 평활화
    df['Smoothed_MA'] = df['MA'].rolling(window=win).mean()

    # 이동평균(Moving Average)를 다시 적용하여 평활화
    df['Smoothed_MA_1'] = df['Smoothed_MA'].rolling(window=win).mean()

    # Date를 열로 가져오고, 레이블 제거
    df = df.reset_index(drop=True)

    # NaN이 포함된 행 제거
    df = df.dropna()
    df = df.drop(['Close', 'MA', 'Smoothed_MA'], axis = 1)
    df.reset_index(drop=True, inplace=True)

    # 'Diff'열 추가
    row_count = df.shape[0]
    for i in range (0, row_count - 1) :
        df.loc[i+1, 'Diff'] =  df.loc[i+1, 'Smoothed_MA_1'] - df.loc[i, 'Smoothed_MA_1'] 

    # NaN이 포함된 행 제거
    df = df.dropna()

    df1 = {'Date': [df.iloc[0,0]],
           'Smoothed_MA_1': [df.iloc[0,1]],
           'Diff': [df.iloc[0,2]]}

    df1 = pd.DataFrame(df1)
    
    for i in range (1, row_count - 2) :
        if df.loc[i+1, 'Diff'] * df.loc[i, 'Diff'] < 0 :
            df1 = df1.append(df.loc[i])

    df1.reset_index(drop=True, inplace=True)
    
    return df1

## 3-12. 주가 추세 데이터 문자열로 치환하는 함수

In [287]:
def chartToString(df, company, m = 6):
    
    #사용자로부터 m개월 인풋 받는 코드 추후 업로드 예정
    month = m
    df1 = df
    row_count = df.shape[0]

    result_str = '{}의 최근 {}개월 간 주가 추세를 분석해드리겠습니다.\n'.format(company, m)

    for i in range(1, row_count-1):
        
        a = df1.loc[i, 'Date'].year
        b = df1.loc[i, 'Date'].month
        c = df1.loc[i, 'Date'].day
        d = df1.loc[i+1, 'Date'].year
        e = df1.loc[i+1, 'Date'].month
        f = df1.loc[i+1, 'Date'].day
        g = abs(round(df1.loc[i, 'Smoothed_MA_1'] - df1.loc[i - 1, 'Smoothed_MA_1'],2))
        h = abs(round(((df1.loc[i, 'Smoothed_MA_1'] - df1.loc[i - 1, 'Smoothed_MA_1']) / df1.loc[i - 1, 'Smoothed_MA_1']) * 100, 2))
        
        if(df1.loc[i, 'Diff'] < 0) :
            result_str += '{}년 {}월 {}일부터 {}년 {}월 {}일까지는 하락 추세입니다.\n'.format(a, b, c, d, e, f)
            result_str += '이 기간 동안 {}의 주가는{}원, {}% 하락합니다.\n'.format(company, g, h)

        elif(df1.loc[i, 'Diff'] > 0):
            result_str += '{}년 {}월 {}일부터 {}년 {}월 {}일까지는 상승 추세입니다.\n'.format(a, b, c, d, e, f)
            result_str += '이 기간 동안 {}의 주가는{}원, {}% 상승합니다.\n'.format(company, g, h)

    return result_str

## 3-13. 주가 예측 함수

In [288]:
def predictStockPrice(company, m = 12):
    
    # 종목 코드 설정 
    code = krx[krx['Name'] == company]['Code'].values[0]

    # 시작 날짜 설정 (현재 날짜로부터 6달 이전)
    # month는 사용자가 설정할 수 있도록 추후 업로드 예정
    month = m
    start_date = datetime.datetime.now() - datetime.timedelta(days=month*30)

    # 주가 데이터 가져오기
    df = fdr.DataReader(code, start_date)

    # 'Date'를 열로 가져오기
    df['Date'] = df.index

    # 'Close'를 제외한 열 제거
    df = df[['Date','Close']]

    # 이동평균(Moving Average) 계산 (기간: 7일) --> 5일, 20일, 60일, 120일
    win = 7 # 사용자의 input값
    df['MA'] = df['Close'].rolling(window=win).mean()

    # 이동평균(Moving Average)를 다시 적용하여 평활화
    df['Smoothed_MA'] = df['MA'].rolling(window=win).mean()

    # 이동평균(Moving Average)를 다시 적용하여 평활화
    df['Smoothed_MA_1'] = df['Smoothed_MA'].rolling(window=win).mean()

    # NaN이 포함된 행 제거
    df = df.dropna()
    df = df.drop(['Date', 'Close', 'MA', 'Smoothed_MA'], axis = 1)
    df = df.rename(columns={'Smoothed_MA_1': 'Close'})
    date = df.index[-1].strftime('%Y-%m-%d')

    data = df
    
    # train_test_split
    train_data, test_data = train_test_split(data, test_size=0.2, shuffle=False)
    
    p = range(0, 3)
    d = range(1, 2)
    q = range(0, 3)
    pdq = list(itertools.product(p, d, q))

    aic = []
    for i in pdq:
        model = SARIMAX(train_data.values, order=i)
        model_fit = model.fit()
        aic.append(round(model_fit.aic, 2))
        
    # Searching Optimal Paramters
    optimal = [(pdq[i], j) for i, j in enumerate(aic) if j == min(aic)]
    optimal

    # 최적의 파라미터를 통한 Optimization
    model_opt = SARIMAX(train_data.values, order=optimal[0][0])
    model_opt_fit = model_opt.fit()
    
    prediction = model_opt_fit.get_forecast(len(test_data))
    predicted_value = prediction.predicted_mean
    predicted_ub = prediction.conf_int()[:, 0]
    predicted_lb = prediction.conf_int()[:, 1]
    predict_index = list(test_data.index)

    # 예측할 날짜 범위 생성
    date_range = pd.date_range(start=date, periods=5, freq='D')

    # 예측할 날짜에 대해 예측 수행
    forecast = model_opt_fit.get_forecast(len(date_range))
    predicted_values = forecast.predicted_mean

    # 예측 결과를 df_2에 저장
    df_2 = pd.DataFrame({'Date': date_range, 'Close': predicted_values})
    df_2.set_index('Date', inplace=True)
    return df_2

## 3-14. 주가 예측 문자열 변환 함수

In [289]:
def predictToString(df, company):
    df_2 = df
    predict_str = "신한AI는 증권투자는 반드시 자기 자신의 판단과 책임하에 하여야 할 것을 당부드립니다.\n"
    predict_str += "신한AI의 시계열 모형을 통해 추후 5일 간 {}의 주가 분석 결과,\n".format(company)

    # 활용 변수
    row_count = df_2.shape[0]
    a = df_2.index[row_count-1].year
    b = df_2.index[row_count-1].month
    c = df_2.index[row_count-1].day

    d = abs(round((df_2['Close'].iloc[row_count-1] - df_2['Close'].iloc[0])))
    e = abs(round(((df_2['Close'].iloc[row_count-1] - df_2['Close'].iloc[0])/df_2['Close'].iloc[row_count-1])*100, 2))

    if df_2['Close'].iloc[row_count-1] - df_2['Close'].iloc[0] > 0:
        predict_str += f"추후 5일간, 삼성전자의 주가는 {d}원만큼 상승하며, 최대 {e}%만큼 상승할 것으로 예측됩니다."
    else:
        predict_str += f"추후 5일간, 삼성전자의 주가는 {d}원만큼 하락하며, 최대 {e}%만큼 하락할 것으로 예측됩니다."

    return predict_str

# 4. 예시 코드 실행

In [290]:
#사용자 어플 로그인 시 오프닝 멘트 재생
#안녕하세요. 고객님의 투자를 도와주는 신한 AI입니다.
stock = None
subprocess.call(['afplay', 'Ment1.wav'])

while True:
    
    try:
        #원하시는 서비스를 말씀해주세요
        subprocess.call(['afplay', 'Ment2.wav'])
        response = voiceRecog(3)

        if "오늘의 뉴스" in response:
            synthesizeText(summary1, "summary1.wav")
            subprocess.call(['afplay', 'summary1.wav'])

        elif "투자 정보" in response:
            found = "no"
            while found == "no":
                
                #원하시는 종목을 말씀해주세요.
                subprocess.call(['afplay', 'Ment5.wav'])
                response2 = voiceRecog(3)
                
                #국내 시장에 상장된 종목 중 일치하는 것 찾기
                for company in companies:
                    if company in response2:
                        stock = company
                        found = "yes"
                        break
                
                #상장 기업 목록에 없는 회사명을 말한 경우
                if found == "no":
                    #말씀하신 종목을 찾을 수 없습니다.
                    subprocess.call(['afplay', 'Ment6.wav'])
            
            while True:
                
                #다음 세가지 서비스 중에서 원하는 것을 말씀해주세요. 1번 최근 기업 뉴스 요약 서비스. 2번 재무 정보 요약 서비스. 
                #3번 주가 추세 분석 서비스. 4번 주가 예측 서비스
                subprocess.call(['afplay', 'Ment4.wav'])
                service  = voiceRecog(3)

                if ("1번" in service) or ("일번" in service):
                    #원하시는 정보를 불러오고 있습니다. 잠시만 기다려주세요
                    subprocess.call(['afplay', 'Ment8.wav'])
                    df_inv1, df_inv2 = crawlingInv(path, stock)
                    #음성 정보를 생성하고 있습니다. 잠시만 기다려주세요
                    subprocess.call(['afplay', 'Ment11.wav'])
#                     summary2 = companyNewsSummarize(df_inv1, stock)
#                     summary22 = companyReportSummarize(df_inv2, stock)
#                     synthesizeText(summary2, "summary2.wav")
                    subprocess.call(['afplay', 'summary2.wav'])
#                     synthesizeText(summary22, "summary22.wav")
                    subprocess.call(['afplay', 'summary22.wav'])

                if ("2번" in service) or ("이번" in service):
                    #원하시는 재무 정보를 말씀해주세요.
                    subprocess.call(['afplay', 'Ment7.wav'])
                    response3 = voiceRecog(5)
                    #원하시는 정보를 불러오고 있습니다. 잠시만 기다려주세요
                    subprocess.call(['afplay', 'Ment8.wav'])
                    df_stock = financeCrawling(path, stock)
                    summary3 = financeString(df_stock, response3)
                    synthesizeText(summary3, "summary3.wav")
                    subprocess.call(['afplay', 'summary3.wav'])

                if ("3번" in service) or ("삼번" in service):
                    df_chart = chartInfo(stock)
                    summary4 = chartToString(df_chart, stock)
                    synthesizeText(summary4, "summary4.wav")
                    subprocess.call(['afplay', 'summary4.wav'])

                if ("4번" in service) or ("사번" in service):
                    df_predict = predictStockPrice(stock)
                    summary5 = predictToString(df_predict, stock)
                    synthesizeText(summary5, "summary5.wav")
                    subprocess.call(['afplay', 'summary5.wav']) 

                #말씀하신 종목의 투자정보 서비스를 계속 이용하시려면 예, 그만 이용하시려면 아니오라고 말씀해주세요.
                subprocess.call(['afplay', 'Ment12.wav'])
                quit_or_not = voiceRecog(3)
                
                if "아니" in quit_or_not:
                    break
                
        else:
            print("outer else")
            #말씀하신 내용을 이해할 수 없습니다.
            subprocess.call(['afplay', 'Ment9.wav'])
        
        #계속해서 신한 AI 서비스를 이용하시려면 예, 서비스를 종료하시려면 아니오라고 말씀해주세요.
        subprocess.call(['afplay', 'Ment3.wav'])
        yesorno = voiceRecog(3)
        
        #아니오라고 대답할 경우 서비스 종료
        if "아니" in yesorno:
            break
            
    except Exception as e:
        print(e)
        #말씀하신 내용을 이해할 수 없습니다.
        subprocess.call(['afplay', 'Ment9.wav'])
        
        
#신한 AI 서비스를 이용해주셔서 감사합니다.
subprocess.call(['afplay', 'Ment10.wav'])

음성을 입력하세요.
음성변환 : 투자 정보
음성을 입력하세요.
음성변환 : 삼성전자
음성을 입력하세요.
음성변환 : 1번 1번


100%|█████████████████████████████████████████████| 3/3 [02:02<00:00, 40.79s/it]


음성을 입력하세요.
음성변환 : 아니오 아니오
음성을 입력하세요.
음성변환 : 아니요 아니요


0